# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vacation_df = pd.read_csv('weatherPy.csv')
vacation_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Breves,100,BR,1584051065,96,-1.68,-50.48,77.05,0.29
1,Vila Velha,75,BR,1584051070,88,-20.33,-40.29,78.01,9.17
2,Mahébourg,75,MU,1584051075,94,-20.41,57.70,79.00,3.36
3,Bredasdorp,0,ZA,1584051245,88,-34.53,20.04,66.20,1.12
4,Mataura,10,NZ,1584051356,89,-46.19,168.86,49.33,0.25


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = vacation_df[["Lat", "Lng"]].astype(float)
weight = vacation_df["Humidity"].astype(float)

vacation_df = vacation_df.dropna()
vacation_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Breves,100,BR,1584051065,96,-1.68,-50.48,77.05,0.29
1,Vila Velha,75,BR,1584051070,88,-20.33,-40.29,78.01,9.17
2,Mahébourg,75,MU,1584051075,94,-20.41,57.70,79.00,3.36
3,Bredasdorp,0,ZA,1584051245,88,-34.53,20.04,66.20,1.12
4,Mataura,10,NZ,1584051356,89,-46.19,168.86,49.33,0.25
...,...,...,...,...,...,...,...,...,...
578,Sibut,99,CF,1584052016,90,5.72,19.07,73.09,3.44
579,Gornopravdinsk,100,RU,1584052018,90,60.05,69.90,34.03,14.16
580,Inhambane,1,MZ,1584052019,81,-23.86,35.38,77.95,8.97
581,Örnsköldsvik,100,SE,1584052020,94,63.29,18.72,32.00,12.75


In [5]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
vacation_df[vacation_df['Wind Speed'] <10]
vacation_df[vacation_df['Cloudiness'] ==0]
vacation_df[(vacation_df['Max Temp'] >70.0) & (vacation_df['Max Temp'] <80.0)]

ideal_weather_df = vacation_df[(vacation_df['Wind Speed'] <10)&(vacation_df['Cloudiness'] ==0)&(vacation_df['Max Temp'] >70.0) & (vacation_df['Max Temp'] <80.0)]
ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
27,Copiapó,0,CL,1584051384,60,-27.37,-70.33,71.60,9.17
130,Mount Isa,0,AU,1584051423,69,-20.73,139.50,75.20,6.93
166,Narasannapeta,0,IN,1584051542,91,18.42,84.05,74.53,2.55
239,Mercedes,0,AR,1584051628,72,-34.65,-59.43,79.00,3.00
274,Yulara,0,AU,1584051666,30,-25.24,130.99,71.60,5.82
293,Sittwe,0,MM,1584051687,68,20.15,92.90,73.45,4.79
301,São Filipe,0,CV,1584051696,58,14.90,-24.50,76.33,7.34
364,Godda,0,IN,1584051770,41,24.83,87.22,75.07,3.69
460,Gweta,0,BW,1584051881,54,-20.18,25.23,76.69,7.29
515,Shakawe,0,BW,1584051946,74,-18.37,21.85,70.93,4.03


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Copiapó,0,CL,1584051384,60,-27.37,-70.33,71.60,9.17
1,Mount Isa,0,AU,1584051423,69,-20.73,139.50,75.20,6.93
2,Narasannapeta,0,IN,1584051542,91,18.42,84.05,74.53,2.55
3,Mercedes,0,AR,1584051628,72,-34.65,-59.43,79.00,3.00
4,Yulara,0,AU,1584051666,30,-25.24,130.99,71.60,5.82
5,Sittwe,0,MM,1584051687,68,20.15,92.90,73.45,4.79
6,São Filipe,0,CV,1584051696,58,14.90,-24.50,76.33,7.34
7,Godda,0,IN,1584051770,41,24.83,87.22,75.07,3.69
8,Gweta,0,BW,1584051881,54,-20.18,25.23,76.69,7.29
9,Shakawe,0,BW,1584051946,74,-18.37,21.85,70.93,4.03


In [8]:
hotel_df = hotel_df.drop(['Cloudiness','Date','Max Temp','Wind Speed'], axis=1)
hotel_df = hotel_df.reset_index(drop=True)

hotel_df

,City,Country,Humidity,Lat,Lng
0,Copiapó,CL,60,-27.37,-70.33
1,Mount Isa,AU,69,-20.73,139.50
2,Narasannapeta,IN,91,18.42,84.05
3,Mercedes,AR,72,-34.65,-59.43
4,Yulara,AU,30,-25.24,130.99
5,Sittwe,MM,68,20.15,92.90
6,São Filipe,CV,58,14.90,-24.50
7,Godda,IN,41,24.83,87.22
8,Gweta,BW,54,-20.18,25.23
9,Shakawe,BW,74,-18.37,21.85


In [9]:
hotelRadius = 5000
hotelKeyword = "hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
        
        "radius": hotelRadius,
        "keyword": hotelKeyword,
        "key": g_key
        }
for ind,row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    
    
    
    try:
        print(f"Closest hotel is {response['results'][0]['name']}.")
        hotel_df.loc[ind,'Hotel Name'] = response['results'][0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

Closest hotel is Hotel Pulmahue.
Closest hotel is Barkly Hotel.
Closest hotel is Sri laxmi lodge.
Closest hotel is Gran Hotel Mercedes.
Closest hotel is Desert Gardens Hotel - Ayers Rock Resort.
Closest hotel is Hotel Memory.
Closest hotel is Hotel Ocean View & Restaurante Seafood.
Closest hotel is HOTEL TUSH TAYA RESIDENCY.
Closest hotel is Gweta Lodge.
Closest hotel is Ditlhapi lodge.
Closest hotel is The Bungalow Hambantota.


In [10]:
hotel_df

,City,Country,Humidity,Lat,Lng,Hotel Name
0,Copiapó,CL,60,-27.37,-70.33,Hotel Pulmahue
1,Mount Isa,AU,69,-20.73,139.50,Barkly Hotel
2,Narasannapeta,IN,91,18.42,84.05,Sri laxmi lodge
3,Mercedes,AR,72,-34.65,-59.43,Gran Hotel Mercedes
4,Yulara,AU,30,-25.24,130.99,Desert Gardens Hotel - Ayers Rock Resort
5,Sittwe,MM,68,20.15,92.90,Hotel Memory
6,São Filipe,CV,58,14.90,-24.50,Hotel Ocean View & Restaurante Seafood
7,Godda,IN,41,24.83,87.22,HOTEL TUSH TAYA RESIDENCY
8,Gweta,BW,54,-20.18,25.23,Gweta Lodge
9,Shakawe,BW,74,-18.37,21.85,Ditlhapi lodge


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
# I could not quite figure this part out.

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)


# Assign the marker layer to a variable
markerS = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markerS)
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)

fig

# Display Map

Figure(layout=FigureLayout(height='420px'))